# Temporal Difference Learning (TD)

The Monte Carlo methods introduced in the previous chapter only learn from complete episodes. This is a major problem because it means we're throwing away information, and we can't use those algorithms for MDPs without terminal states.

TD methods fix these problems by using bootstrapping, thus combining Monte Carlo with Dynamic Programming. This has the advantage that TD methods can learn online during an episode.

## Observable environment

This is essentially the same as in the previous chapter. We can only sample from the environment, but don't know all of its dynamics.

In [32]:
import numpy as np

class ObservableEnvironment:
    def get_states(self):
        """
        Return the set of possible states
        """
        pass
    
    def get_possible_actions(self, state):
        """
        Returns the actions that can be taken from the given state
        """
        pass
    
    def execute_action(self, state, action):
        """
        Returns the new state and the given reward. This does not have to
        be deterministic
        """
        pass
    
    def is_terminal_state(self, state):
        """
        Returns a boolean indicating whether the state is terminal
        """
        pass
    
    def sample(self, policy, state):
        """
        Follows the policy until a terminal state is reached.
        Returns a list of states, actions, and the rewards they gave
        """
        
        result = []
                
        while not self.is_terminal_state(state):
            actions = self.get_possible_actions(state)
            ps = [policy[(state, action)] for action in actions]
            
            action = np.random.choice(actions, p=ps)
            
            new_state, reward = self.execute_action(state, action)
            
            result.append((state, action, reward))
            state = new_state
        
        return result

## Windy GridWorld

This is a standard GridWorld, except that there's also wind pushing the player upwards.

In [179]:
from itertools import product

class WindyGridWorld(ObservableEnvironment):
    def __init__(self, rewards, wind):
        self.rewards = rewards
        self.wind = wind
        
        n, m = rewards.shape
        self.states = list(product(range(n), range(m)))
        
        self.max_down = n - 1
        self.max_right = m - 1
        
        self.UP = "UP"
        self.DOWN = "DOWN"
        self.LEFT = "LEFT"
        self.RIGHT = "RIGHT"
        
        self.ACTIONS = [self.UP, self.DOWN, self.LEFT, self.RIGHT]
        
    def get_states(self):
        return self.states
    
    def get_possible_actions(self, state):
        i, j = state
        
        actions = []
        
        if i > 0:
            actions.append(self.UP)
            
        if i < self.max_down:
            actions.append(self.DOWN)
        
        if j > 0:
            actions.append(self.LEFT)
        
        if j < self.max_right:
            actions.append(self.RIGHT)
            
        return actions
    
    def execute_action(self, state, action):
        i, j = state        
        
        if action == self.UP:
            i -= 1
            
        if action == self.DOWN:
            i += 1
            
        if action == self.LEFT:
            j -= 1
            
        if action == self.RIGHT:
            j += 1
            
        i -= self.wind[j]
            
        i = max(0, min(i, self.max_down))
        j = max(0, min(j, self.max_right))
        
        return (i, j), self.rewards[(i, j)] - 1
    
    def is_terminal_state(self, state):
        # In this GridWorld we terminate once the reward is reached
        
        i, j = state
        return self.rewards[i, j] > 0

Some code to visualize the results:

In [180]:
def draw_policy(agent, n, m):
    lookup = {
        "UP": u"↑",
        "DOWN": u"↓",
        "LEFT": u"←",
        "RIGHT": u"→"
    }
    
    result = np.zeros((n, m))
    
    for i in range(n):
        result = ""
        
        for j in range(m):
            state = (i, j)
            actions = agent.env.get_possible_actions(state)      
            num_actions = len(actions)

            QA = { possible_action: agent.Q[(state, possible_action)] for possible_action in actions }
            best_action = max(QA.items(), key=itemgetter(1))[0]
            
            result += lookup[best_action]
        
        print result

We'll evaluate all algorithms on the following environment:

In [147]:
R = np.array([
    [0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0]
])

wind = [0, 0, 1, 1, 1, 0]

env = WindyGridWorld(R, wind)

## Sarsa

SARSA = **S**tate, **A**ction, **R**eward, Next **S**tate, Next **A**ction

The value for the current state and action (SA) is updated by measuring the difference between the current guess and the sum of the observed reward (R) and the estimated value of the next state, action pair (SA).

This is an on-policy algorithm because we're continually trying to estimate the action values and then always choose an $\epsilon$-greedy policy according to those action values.

In [198]:
from random import choice
from operator import itemgetter

class SarsaAgent:
    def __init__(self, env, discount_factor=1., alpha=.4, epsilon=.05):
        self.env = env
        self.discount_factor = discount_factor
        self.alpha = alpha
        self.epsilon = epsilon
        
        self.Q = {}
        self.policy = {}
        
        for state in env.get_states():
            actions = env.get_possible_actions(state)
            num_actions = len(actions)
            
            for action in actions:
                self.Q[(state, action)] = 0.
                self.policy[(state, action)] = 1. / num_actions
    
    def learn(self, num_samples, initial_state):
        for _ in range(num_samples):
            S = initial_state
            A = self._choose_action(S)
            
            while not self.env.is_terminal_state(S):
                S, A = self._inner_learn(S, A)
                self._update_policy()
                
    def _inner_learn(self, S, A):
        S_next, R = self.env.execute_action(S, A)
        A_next = self._choose_action(S_next)

        # SA R SA
        self.Q[(S, A)] += self.alpha * (R + self.discount_factor * self.Q[(S_next, A_next)] - self.Q[(S, A)])

        return S_next, A_next
                
    def _choose_action(self, S):
        actions = self.env.get_possible_actions(S)
        
        ps = [self.policy[(S, A)] for A in actions]
        A = np.random.choice(actions, p=ps)
        
        return A
    
    def _update_policy(self):
        for state in self.env.get_states():
            actions = self.env.get_possible_actions(state)      
            num_actions = len(actions)
            
            best_action = self._find_best_action(state)
            
            for action in actions:
                self.policy[(state, action)] = self.epsilon / num_actions
                
            self.policy[(state, best_action)] += 1 - self.epsilon
            
    def _find_best_action(self, state):
        QA = { action: self.Q[(state, action)] for action in self.env.get_possible_actions(state) }
        return max(QA.items(), key=itemgetter(1))[0]

In [215]:
sarsa_agent = SarsaAgent(env)
sarsa_agent.learn(num_samples=10000, initial_state=(0, 0))

In [216]:
draw_policy(sarsa_agent, 3, 6)

→→→→→↓
→→→↓↓↓
→→→→→←


By looking at the reward matrix and the configured wind, it's easy to see that this policy is optimal.

In [183]:
R

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0]])

In [185]:
wind

[0, 0, 1, 1, 1, 0]

## Q-Learning

Q-learning is extremely similar to Sarsa, the only difference is that we use a different second A. In Sarsa we just follow the policy to choose the next action (on-policy). In Q-learning we just the action that's currently considered to be the best, which makes Q-learning an off-policy approach.

In [208]:
class QLearningAgent(SarsaAgent):
    def _inner_learn(self, S, A):
        A = self._choose_action(S)
        S_next, R = self.env.execute_action(S, A)
        
        A_best = self._find_best_action(S_next)
        self.Q[(S, A)] += self.alpha * (R + self.discount_factor * self.Q[(S_next, A_best)] - self.Q[(S, A)])
        
        return S_next, A

In [218]:
q_agent = QLearningAgent(env)
q_agent.learn(num_samples=1000, initial_state=(0, 0))

This agent learns the same policy as the Sarsa Agent, but interestingly it does so much faster.

In [219]:
draw_policy(q_agent, 3, 6)

→→→→→↓
→→→↓↓↓
→→→→→←


## Expected Sarsa

Again, this is just like Sarsa, except that we replace the second A. This time it's replaced by the expected value over all subsequent S, A pairs.

In [221]:
class ExpectedSarsaAgent(SarsaAgent):
    def _inner_learn(self, S, A):
        S_next, R = self.env.execute_action(S, A)
        A_next = self._choose_action(S_next)
        
        expected_next_value = 0
        for action in self.env.get_possible_actions(S_next):
            p = self.policy[(S_next, action)]
            Q = self.Q[(S_next, action)]
            
            expected_next_value += p * Q

        # SA R E(SA)
        self.Q[(S, A)] += self.alpha * (R + self.discount_factor * expected_next_value - self.Q[(S, A)])

        return S_next, A_next

In [226]:
expected_sarsa_agent = QLearningAgent(env)
expected_sarsa_agent.learn(num_samples=12000, initial_state=(0, 0))

We arrive at the same policy, but it actually takes a bit longer than normal Sarsa. This can most probably be improved by choosing a more sensible $\epsilon$.

In [227]:
draw_policy(expected_sarsa_agent, 3, 6)

→→→→→↓
→→→↓↓↓
→→→→→←
